In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv
/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv
/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv
/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=c8e1266e4ca1fd2ca0ae252bf943d42f4762fea25a829a2f559ed17609fab767
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import *
import time

In [51]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [17]:
#spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [20]:
!rm -r /kaggle/working/*

Original Code

Bucket Size 400

In [21]:
start_time1 = time.time()
parkViolations_2015 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
parkViolations_2016 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
new_column_name_list= list(map(lambda x: x.replace(" ", "_"), parkViolations_2015.columns))
parkViolations_2015 = parkViolations_2015.toDF(*new_column_name_list)
parkViolations_2015 = parkViolations_2015.filter(parkViolations_2015.Plate_Type == "COM").filter(parkViolations_2015.Vehicle_Year == "2001")
parkViolations_2016 = parkViolations_2016.toDF(*new_column_name_list)
parkViolations_2016 = parkViolations_2016.filter(parkViolations_2016.Plate_Type == "COM").filter(parkViolations_2016.Vehicle_Year == "2001")
parkViolations_2015.write.mode("overwrite").bucketBy(400, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2015")
parkViolations_2016.write.mode("overwrite").bucketBy(400, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2016")
parkViolations_2015_tbl = spark.read.table("parkViolations_bkt_2015")
parkViolations_2016_tbl = spark.read.table("parkViolations_bkt_2016")
start_time2 = time.time()
joinDF = parkViolations_2015_tbl.join(parkViolations_2016_tbl, (parkViolations_2015_tbl.Plate_Type ==  parkViolations_2016_tbl.Plate_Type) & (parkViolations_2015_tbl.Vehicle_Year ==  parkViolations_2016_tbl.Vehicle_Year) , "inner").select(parkViolations_2015_tbl["Summons_Number"], parkViolations_2016_tbl["Issue_Date"])
time_taken_by_join = time.time() - start_time2
joinDF.explain()
joinDF.count()
#joinDF.write.format("com.databricks.spark.csv").option("header", True).mode("overwrite").save("/kaggle/working/bkt_joined_df.csv")
time_taken_for_all = time.time() - start_time1
print("Time Taken only by Join:", time_taken_by_join, "seconds")
print("Time Taken for the whole code to run:", time_taken_for_all, "seconds")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Summons_Number#2976, Issue_Date#3082]
   +- SortMergeJoin [Vehicle_Year#3011, Plate_Type#2979], [Vehicle_Year#3113, Plate_Type#3081], Inner
      :- Sort [Vehicle_Year#3011 ASC NULLS FIRST, Plate_Type#2979 ASC NULLS FIRST], false, 0
      :  +- Filter (isnotnull(Plate_Type#2979) AND isnotnull(Vehicle_Year#3011))
      :     +- FileScan parquet spark_catalog.default.parkviolations_bkt_2015[Summons_Number#2976,Plate_Type#2979,Vehicle_Year#3011] Batched: true, Bucketed: true, DataFilters: [isnotnull(Plate_Type#2979), isnotnull(Vehicle_Year#3011)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/kaggle/working/spark-warehouse/parkviolations_bkt_2015], PartitionFilters: [], PushedFilters: [IsNotNull(Plate_Type), IsNotNull(Vehicle_Year)], ReadSchema: struct<Summons_Number:string,Plate_Type:string,Vehicle_Year:string>, SelectedBucketsCount: 400 out of 400
      +- Sort [Vehicle_Year#3113 ASC NULLS FIRST, Plate_Typ

Time Taken only by Join: 0.03740334510803223 seconds
Time Taken for the whole code to run: 98.46608924865723 seconds


Bucket Size 200

In [22]:
start_time1 = time.time()
parkViolations_2015 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
parkViolations_2016 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
new_column_name_list= list(map(lambda x: x.replace(" ", "_"), parkViolations_2015.columns))
parkViolations_2015 = parkViolations_2015.toDF(*new_column_name_list)
parkViolations_2015 = parkViolations_2015.filter(parkViolations_2015.Plate_Type == "COM").filter(parkViolations_2015.Vehicle_Year == "2001")
parkViolations_2016 = parkViolations_2016.toDF(*new_column_name_list)
parkViolations_2016 = parkViolations_2016.filter(parkViolations_2016.Plate_Type == "COM").filter(parkViolations_2016.Vehicle_Year == "2001")
parkViolations_2015.write.mode("overwrite").bucketBy(200, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2015")
parkViolations_2016.write.mode("overwrite").bucketBy(200, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2016")
parkViolations_2015_tbl = spark.read.table("parkViolations_bkt_2015")
parkViolations_2016_tbl = spark.read.table("parkViolations_bkt_2016")
start_time2 = time.time()
joinDF = parkViolations_2015_tbl.join(parkViolations_2016_tbl, (parkViolations_2015_tbl.Plate_Type ==  parkViolations_2016_tbl.Plate_Type) & (parkViolations_2015_tbl.Vehicle_Year ==  parkViolations_2016_tbl.Vehicle_Year) , "inner").select(parkViolations_2015_tbl["Summons_Number"], parkViolations_2016_tbl["Issue_Date"])
time_taken_by_join = time.time() - start_time2
joinDF.explain()
joinDF.count()
#joinDF.write.format("com.databricks.spark.csv").option("header", True).mode("overwrite").save("/kaggle/working/bkt_joined_df.csv")
time_taken_for_all = time.time() - start_time1
print("Time Taken only by Join:", time_taken_by_join, "seconds")
print("Time Taken for the whole code to run:", time_taken_for_all, "seconds")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Summons_Number#4145, Issue_Date#4251]
   +- SortMergeJoin [Vehicle_Year#4180, Plate_Type#4148], [Vehicle_Year#4282, Plate_Type#4250], Inner
      :- Sort [Vehicle_Year#4180 ASC NULLS FIRST, Plate_Type#4148 ASC NULLS FIRST], false, 0
      :  +- Filter (isnotnull(Plate_Type#4148) AND isnotnull(Vehicle_Year#4180))
      :     +- FileScan parquet spark_catalog.default.parkviolations_bkt_2015[Summons_Number#4145,Plate_Type#4148,Vehicle_Year#4180] Batched: true, Bucketed: true, DataFilters: [isnotnull(Plate_Type#4148), isnotnull(Vehicle_Year#4180)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/kaggle/working/spark-warehouse/parkviolations_bkt_2015], PartitionFilters: [], PushedFilters: [IsNotNull(Plate_Type), IsNotNull(Vehicle_Year)], ReadSchema: struct<Summons_Number:string,Plate_Type:string,Vehicle_Year:string>, SelectedBucketsCount: 200 out of 200
      +- Sort [Vehicle_Year#4282 ASC NULLS FIRST, Plate_Typ

Time Taken only by Join: 0.03660011291503906 seconds
Time Taken for the whole code to run: 88.18751358985901 seconds


Bucket Size 800

In [23]:
start_time1 = time.time()
parkViolations_2015 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
parkViolations_2016 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
new_column_name_list= list(map(lambda x: x.replace(" ", "_"), parkViolations_2015.columns))
parkViolations_2015 = parkViolations_2015.toDF(*new_column_name_list)
parkViolations_2015 = parkViolations_2015.filter(parkViolations_2015.Plate_Type == "COM").filter(parkViolations_2015.Vehicle_Year == "2001")
parkViolations_2016 = parkViolations_2016.toDF(*new_column_name_list)
parkViolations_2016 = parkViolations_2016.filter(parkViolations_2016.Plate_Type == "COM").filter(parkViolations_2016.Vehicle_Year == "2001")
parkViolations_2015.write.mode("overwrite").bucketBy(800, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2015")
parkViolations_2016.write.mode("overwrite").bucketBy(800, "Vehicle_Year", "plate_type").saveAsTable("parkViolations_bkt_2016")
parkViolations_2015_tbl = spark.read.table("parkViolations_bkt_2015")
parkViolations_2016_tbl = spark.read.table("parkViolations_bkt_2016")
start_time2 = time.time()
joinDF = parkViolations_2015_tbl.join(parkViolations_2016_tbl, (parkViolations_2015_tbl.Plate_Type ==  parkViolations_2016_tbl.Plate_Type) & (parkViolations_2015_tbl.Vehicle_Year ==  parkViolations_2016_tbl.Vehicle_Year) , "inner").select(parkViolations_2015_tbl["Summons_Number"], parkViolations_2016_tbl["Issue_Date"])
time_taken_by_join = time.time() - start_time2
joinDF.explain()
joinDF.count()
#joinDF.write.format("com.databricks.spark.csv").option("header", True).mode("overwrite").save("/kaggle/working/bkt_joined_df.csv")
time_taken_for_all = time.time() - start_time1
print("Time Taken only by Join:", time_taken_by_join, "seconds")
print("Time Taken for the whole code to run:", time_taken_for_all, "seconds")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Summons_Number#5314, Issue_Date#5420]
   +- SortMergeJoin [Vehicle_Year#5349, Plate_Type#5317], [Vehicle_Year#5451, Plate_Type#5419], Inner
      :- Sort [Vehicle_Year#5349 ASC NULLS FIRST, Plate_Type#5317 ASC NULLS FIRST], false, 0
      :  +- Filter (isnotnull(Plate_Type#5317) AND isnotnull(Vehicle_Year#5349))
      :     +- FileScan parquet spark_catalog.default.parkviolations_bkt_2015[Summons_Number#5314,Plate_Type#5317,Vehicle_Year#5349] Batched: true, Bucketed: true, DataFilters: [isnotnull(Plate_Type#5317), isnotnull(Vehicle_Year#5349)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/kaggle/working/spark-warehouse/parkviolations_bkt_2015], PartitionFilters: [], PushedFilters: [IsNotNull(Plate_Type), IsNotNull(Vehicle_Year)], ReadSchema: struct<Summons_Number:string,Plate_Type:string,Vehicle_Year:string>, SelectedBucketsCount: 800 out of 800
      +- Sort [Vehicle_Year#5451 ASC NULLS FIRST, Plate_Typ

Time Taken only by Join: 0.03946566581726074 seconds
Time Taken for the whole code to run: 91.12543487548828 seconds


In [38]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1)

In [40]:
#BroadcastJoin

start_time1 = time.time()
parkViolations_2015 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
parkViolations_2016 = spark.read.option("header", True).csv("/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
parkViolations_2015 = parkViolations_2015.withColumnRenamed("Plate Type", "plateType") # simple column rename for easier joins
parkViolations_2016 = parkViolations_2016.withColumnRenamed("Plate Type", "plateType")

parkViolations_2015_COM = parkViolations_2015.filter(parkViolations_2015.plateType == "COM").select("plateType", "Summons Number").distinct()
parkViolations_2016_COM = parkViolations_2016.filter(parkViolations_2016.plateType == "COM").select("plateType", "Issue Date").distinct()
parkViolations_2015_COM.cache()
parkViolations_2016_COM.cache()
parkViolations_2015_COM.count() # will cause parkViolations_2015_COM to be cached
parkViolations_2016_COM.count() # will cause parkViolations_2016_COM to be cached
start_time2 = time.time()
joinDF = parkViolations_2015_COM.join(parkViolations_2016_COM.hint("broadcast"), parkViolations_2015_COM.plateType ==  parkViolations_2016_COM.plateType, "inner").select(parkViolations_2015_COM["Summons Number"], parkViolations_2016_COM["Issue Date"])
time_taken_by_join = time.time() - start_time2
joinDF.explain() # you will see BroadcastHashJoin
joinDF.count()
#joinDF.write.format("com.databricks.spark.csv").option("header", True).mode("overwrite").save("/user/kfnu/output/joined_df")
#exit()
time_taken_for_all = time.time() - start_time1
print("Time Taken only by Join:", time_taken_by_join, "seconds")
print("Time Taken for the whole code to run:", time_taken_for_all, "seconds")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Summons Number#9399, Issue Date#9522]
   +- BroadcastHashJoin [plateType#9620], [plateType#9673], Inner, BuildRight, false
      :- Filter isnotnull(plateType#9620)
      :  +- InMemoryTableScan [plateType#9620, Summons Number#9399], [isnotnull(plateType#9620)]
      :        +- InMemoryRelation [plateType#9620, Summons Number#9399], StorageLevel(disk, memory, deserialized, 1 replicas)
      :              +- *(2) HashAggregate(keys=[plateType#9620, Summons Number#9399], functions=[])
      :                 +- Exchange hashpartitioning(plateType#9620, Summons Number#9399, 200), ENSURE_REQUIREMENTS, [plan_id=2316]
      :                    +- *(1) HashAggregate(keys=[plateType#9620, Summons Number#9399], functions=[])
      :                       +- *(1) Project [Plate Type#9402 AS plateType#9620, Summons Number#9399]
      :                          +- *(1) Filter (isnotnull(Plate Type#9402) AND (Plate Type#9402 = C

Time Taken only by Join: 0.058622121810913086 seconds
Time Taken for the whole code to run: 181.35094141960144 seconds
